In [137]:
# SELFACT_CBC.IPYNB - A simple case of resource compeition CBC applied to a self-activating gene, with a basic model
# By Kirill Sechkar

# PACKAGE IMPORTS
import numpy as np
import jax
import jax.numpy as jnp
from bokeh import plotting as bkplot, models as bkmodels, layouts as bklayouts, io as bkio
from bokeh.colors import RGB as bkRGB
import matplotlib as mpl, matplotlib.pyplot as plt
from bokeh import io as bkio
from bokeh.colors import RGB as bkRGB

# SOLVER AND CONTROLLER IMPORTS
import common.controllers as ctrls
import common.reference_switchers as refsws
import common.ode_solvers as odesols

# BIFURCATION ANALYSIS, PROBE CHARACTERISATION AND PREDICTION TOOLS
from common import selfact_an_bif as an_tools
from common.selfact_jointexp import *
from common.probe_char_tools import *

# CELL AND CIRCUIT MODEL IMPORTS
from basic_model import *
import basic_genetic_modules as gms

# set up jax
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)

# set up bokeh
bkio.reset_output()
bkplot.output_notebook()

# set up matplotlib
%matplotlib widget

Loading BokehJS ...

In [138]:
# INITIALISE THE MODEL, ITS PARAMETERS AND INITIAL CONDITIONS
model_auxil = ModelAuxiliary()  # auxiliary tools for simulating the model and plotting simulation outcomes
par = model_auxil.default_params()  # get default parameter values
init_conds = model_auxil.default_init_conds(par)  # get default initial conditions

In [139]:
# DEFINE CIRCUIT PARAMETERS TO BE CONSIDERED
circ_par={}

# probe parameters
circ_par['q_ta'] = 45.0  # RC factor for the transcription activation factor
circ_par['q_b'] = 6e4  # RC factor for the burdensome gene of the probe
circ_par['mu_b'] = 1 / (13.6 / 60)
circ_par['baseline_tai-dna'] = 0.01

# cifi (chemically induced gene with a fixed input) parameters
circ_par['q_ta2'] = 30.0  # RC factor for the transcription activation factor
circ_par['q_ta3'] = 30.0  # RC factor for the transcription activation factor
circ_par['q_ta4'] = 120.0  # RC factor for the transcription activation factor
circ_par['q_b2'] = 75*120.0 # RC factor for the burdensome gene of the probe
circ_par['mu_b2'] = 1 / (13.6 / 60)
# regulation by ta 2
circ_par['K_ta2-i'] = 100
circ_par['K_ta2i-dna'] = 100
circ_par['baseline_ta2i-dna'] = 0.00
circ_par['eta_ta2i-dna'] = 5.0
circ_par['u2']=8.0
# regulation by ta 3
circ_par['K_ta3-i'] = 100
circ_par['K_ta3i-dna'] = 100
circ_par['baseline_ta3i-dna'] = 0.00
circ_par['eta_ta3i-dna'] = 5.0
circ_par['u3']=16.0
# regulation by ta4
circ_par['K_ta4-i'] = 100
circ_par['K_ta4i-dna'] = 100
circ_par['baseline_ta4i-dna'] = 0.00
circ_par['eta_ta4i-dna'] = 5.0
circ_par['u4']=5.0

In [140]:
I2=circ_par['u2']/(circ_par['u2']+circ_par['K_ta2-i'])
print('I2 = '+str(I2))
I3=circ_par['u3']/(circ_par['u3']+circ_par['K_ta3-i'])
print('I3 = '+str(I3))
I4=circ_par['u4']/(circ_par['u4']+circ_par['K_ta4-i'])
print('I4 = '+str(I4))

I2 = 0.07407407407407407
I3 = 0.13793103448275862
I4 = 0.047619047619047616


In [141]:
# CALCULATE CORRESPONDING NORMALISED RC FACTORS
cellvars={}
cellvars['Q_ta_max']=circ_par['q_ta']/(par['q_r']+par['q_o'])
cellvars['Q_b_max']=circ_par['q_b']/(par['q_r']+par['q_o'])
cellvars['Q_ta2_max']=circ_par['q_ta2']/(par['q_r']+par['q_o'])
cellvars['Q_ta3_max']=circ_par['q_ta3']/(par['q_r']+par['q_o'])
cellvars['Q_b2_max']=circ_par['q_b2']/(par['q_r']+par['q_o'])


In [142]:
# SIMULATE CBC - INITIALISE AND PARAMETERISE THE CELL MODEL

# add reference tracker switcher
model_par_with_refswitch, ref_switcher = model_auxil.add_reference_switcher(par,
                                                                            # cell model parameters
                                                                            refsws.timed_switching_initialise,
                                                                            # function initialising the reference switcher
                                                                            refsws.timed_switching_switch
                                                                            # function switching the references to be tracked
                                                                            )

# load synthetic genetic modules and the controller
odeuus_complete, \
    module1_F_calc, module2_F_calc, \
    module1_specterms, module2_specterms, \
    controller_action, controller_update, \
    par, init_conds, controller_memo0, \
    synth_genes_total_and_each, synth_miscs_total_and_each, \
    controller_memos, controller_dynvars, controller_ctrledvar, \
    modules_name2pos, modules_styles, controller_name2pos, controller_styles, \
    module1_v_with_F_calc, module2_v_with_F_calc = model_auxil.add_modules_and_controller(
    # module 1
    gms.cifi_initialise,  # function initialising the circuit
    gms.cifi_ode,  # function defining the circuit ODEs
    gms.cifi_F_calc, # function calculating the circuit genes' transcription regulation functions
    gms.cifi_specterms, # function calculating the circuit genes effective mRNA levels (due to possible co-expression from the same operons)
    # module 2
    gms.cicc_initialise,  # function initialising the circuit
    gms.cicc_ode,  # function defining the circuit ODEs
    gms.cicc_F_calc,    # function calculating the circuit genes' transcription regulation functions
    gms.cicc_specterms,  # function calculating the circuit genes effective mRNA levels (due to possible co-expression from the same operons)
    # controller
    ctrls.pichem_initialise,  # function initialising the controller
    ctrls.pichem_action,  # function calculating the controller action
    ctrls.pichem_ode,  # function defining the controller ODEs
    ctrls.pichem_update,  # function updating the controller based on measurements
    # cell model parameters and initial conditions
    model_par_with_refswitch, init_conds)

# unpack the synthetic genes and miscellaneous species lists
synth_genes = synth_genes_total_and_each[0]
module1_genes = synth_genes_total_and_each[1]
module2_genes = synth_genes_total_and_each[2]
synth_miscs = synth_miscs_total_and_each[0]
module1_miscs = synth_miscs_total_and_each[1]
module2_miscs = synth_miscs_total_and_each[2]

par.update(circ_par)

In [143]:
# SIMULATE CBC - SET CONTROLLER AND SIMULATION PARAMETER

controller_ctrledvar='b2_mature'

# reference p_b2 levels
cbc_refs=np.linspace(0, 361204.0, 30)
cbc_refs=np.flip(cbc_refs)

# simulation parameters

# switching between references
par['t_burn_in']=24.0
cbc_exp_duration=240.0
par['t_switch_ref']=(cbc_exp_duration/len(cbc_refs))

tf = (0.0, par['t_burn_in']+cbc_exp_duration)  # simulation time
meastimestep = 0.1  # hours


# PI controller parameters - gains negative as more probe induction => less p_b2
par['Kp'] = -0.00005
# par['Kp'] = -0.001
par['Ki'] = 0#par['Kp']/10

# initial control action
u0=0.0

control_delay=0.0

In [144]:
# SIMULATE CBC - RUN THE SIMULATION

# set up the ODE solver
ode_solver, us_size = odesols.create_euler_solver(odeuus_complete,
                                                  control_delay=control_delay,
                                                  meastimestep=meastimestep,
                                                  euler_timestep=1e-5)

# solve ODE
timer= time.time()
ts_jnp, xs_jnp,\
    ctrl_memorecord_jnp, uexprecord_jnp, \
    refrecord_jnp  = ode_sim(par,   # model parameters
                             ode_solver,    # ODE solver for the cell with the synthetic gene circuit
                             odeuus_complete,    # ODE function for the cell with the synthetic gene circuit and the controller (also gives calculated and experienced control actions)
                             controller_ctrledvar,    # name of the variable read and steered by the controller
                             controller_update, controller_action,   # function for updating the controller memory and calculating the control action
                             model_auxil.x0_from_init_conds(init_conds,
                                                                par,
                                                                synth_genes, synth_miscs, controller_dynvars,
                                                                modules_name2pos,
                                                                controller_name2pos),   # initial condition VECTOR
                             controller_memo0,  # initial controller memory record
                             u0,    # initial control action, applied before any measurement-informed actions reach the system
                             (len(synth_genes), len(module1_genes), len(module2_genes)),    # number of synthetic genes
                             (len(synth_miscs), len(module1_miscs), len(module2_miscs)),    # number of miscellaneous species
                             modules_name2pos, controller_name2pos, # dictionaries mapping gene names to their positions in the state vector
                             model_auxil.synth_gene_params_for_jax(par, synth_genes),   # synthetic gene parameters in jax.array form
                             tf, meastimestep,  # simulation time frame and measurement time step
                             control_delay,  # delay before control action reaches the system
                             us_size,  # size of the control action record needed
                             cbc_refs, ref_switcher,  # reference values and reference switcher
                             )

# convert simulation results to numpy arrays
ts = np.array(ts_jnp)
xs = np.array(xs_jnp)
ctrl_memorecord = np.array(ctrl_memorecord_jnp)
uexprecord = np.array(uexprecord_jnp)
refrecord= np.array(refrecord_jnp)

In [145]:
# PLOT - SYNTHETIC CIRCUITS AND CONTROLLER
# plot synthetic circuit concentrations
mRNA_fig, prot_fig, misc_fig = model_auxil.plot_circuit_concentrations(ts, xs,
                                                                          par, synth_genes, synth_miscs,
                                                                          modules_name2pos,
                                                                          modules_styles)  # plot simulation results
# plot synthetic circuit regulation functions
F_fig = model_auxil.plot_circuit_regulation(ts, xs, # time points and state vectors
                                                ctrl_memorecord, uexprecord,    # controller memory and experienced control actions records
                                                refrecord,  # reference tracker records
                                                module1_F_calc, module2_F_calc, # transcription regulation functions for both modules
                                                controller_action, # control action calculator
                                                par, # model parameters
                                                synth_genes_total_and_each,     # list of synthetic genes - total and for each module
                                                synth_miscs_total_and_each,     # list of synthetic miscellaneous species - total and for each module
                                                modules_name2pos,   # dictionary mapping gene names to their positions in the state vector
                                                module1_specterms, module2_specterms, # functions calculating effective mRNA levels for both modules
                                                controller_name2pos, # dictionary mapping controller species to their positions in the state vector
                                                modules_styles)  # plot simulation results
# plot controller memory, dynamic variables and actions
ctrl_ref_fig, ctrl_memo_fig, ctrl_dynvar_fig, ctrl_u_fig = model_auxil.plot_controller(ts, xs,
                                                                                        ctrl_memorecord, uexprecord, # controller memory and experienced control actions records
                                                                                        refrecord, # reference tracker records
                                                                                        controller_memos, controller_dynvars,
                                                                                        controller_ctrledvar,
                                                                                        controller_action, controller_update,
                                                                                        par,
                                                                                        synth_genes, synth_miscs,
                                                                                        modules_name2pos,
                                                                                        module1_specterms, module2_specterms,
                                                                                        controller_name2pos,
                                                                                        controller_styles,
                                                                                        u0, control_delay)
bkplot.show(bklayouts.grid([[mRNA_fig, prot_fig, misc_fig],
                            [F_fig, None, ctrl_ref_fig],
                            [ctrl_memo_fig, ctrl_dynvar_fig, ctrl_u_fig]]))

In [146]:
# plot ta2 concentrations
ta2_fig = bkplot.figure(
    frame_width=250,
    frame_height=150,
    x_axis_label="Time [h]",
    y_axis_label="Prot. conc. [nM]",
    x_range=(0, ts[-1] - par['t_burn_in']),
    # y_range=(0, 1.2e6),
    tools="box_zoom,pan,hover,reset,save"
)
ta2_fig.output_backend = 'svg'

# plot the module-of-interest outputs
ta2_fig.line(x=ts - par['t_burn_in'], y=xs[:, modules_name2pos['p_ta2']]*I2,
               line_width=1.5, color='#de3163ff',
               legend_label='p_ta2*I2')
ta2_fig.line(x=ts - par['t_burn_in'], y=xs[:, modules_name2pos['p_ta3']]*I3,
               line_width=1.5, color='dodgerblue',
               legend_label='p_ta3*I3')


# legend formatting
ta2_fig.legend.location = "top_left"
ta2_fig.legend.label_text_font_size = "6pt"
ta2_fig.legend.padding = 0
ta2_fig.legend.margin = 1
ta2_fig.legend.spacing = 3
ta2_fig.legend.glyph_width = 3
ta2_fig.legend.orientation = "horizontal"
ta2_fig.legend.click_policy = 'hide'

# axis formatting
# ta2_fig.yaxis.formatter = bkmodels.PrintfTickFormatter(format="%4.1e")
# ta2_fig.yaxis.ticker = bkmodels.FixedTicker(ticks=[0.0, 4e5, 8e5, 1.2e6])
ta2_fig.xaxis.axis_label_text_font_size = "6pt"
ta2_fig.yaxis.axis_label_text_font_size = "6pt"
ta2_fig.xaxis.axis_label_text_color = "black"
ta2_fig.yaxis.axis_label_text_color = "black"
ta2_fig.xaxis.major_label_text_font_size = "6pt"
ta2_fig.yaxis.major_label_text_font_size = "6pt"
ta2_fig.xaxis.major_label_text_color = "black"
ta2_fig.yaxis.major_label_text_color = "black"

bkplot.show(ta2_fig)

In [147]:
# GET THE CALCULATED CONTROL ACTIONS FOR THE CBC TRAJECTORY

# get the calculated control actions
ucalcrecord = model_auxil.get_u_calc(ts, xs, ctrl_memorecord, refrecord,
                                         controller_action,
                                         par,
                                         synth_genes, synth_miscs,
                                         modules_name2pos,
                                         module1_specterms, module2_specterms,
                                         controller_name2pos,
                                         controller_ctrledvar)

In [148]:
# GET THE (REAL) BURDENS EXPERIENCED AND IMPOSED BY THE SELF-ACTIVATING SWITCH

Fs_cifi, Fs_probe = model_auxil.get_Fs(ts, xs,  # time points and state vectors
                                          uexprecord,  # experienced control actions records
                                          module1_F_calc, module2_F_calc, # transcription regulation functions for both modules
                                          par,  # model parameters
                                          synth_genes_total_and_each,   # list of synthetic genes - total and for each module
                                          module1_specterms, module2_specterms, # functions calculating effective mRNA levels for both modules
                                          modules_name2pos # dictionary mapping gene names to their positions in the state vector
                                          )

# get the burdens experienced by the self-activating switch
Q_probes=Fs_probe[:, modules_name2pos['F_ta']]*cellvars['Q_ta_max'] + Fs_probe[:, modules_name2pos['F_b']]*cellvars['Q_b_max']

# get the burdens imposed by the self-activating switch
Q_cifis=Fs_cifi[:, modules_name2pos['F_ta2']]*cellvars['Q_ta2_max'] + Fs_cifi[:, modules_name2pos['F_ta3']]*cellvars['Q_ta3_max']+Fs_cifi[:, modules_name2pos['F_b2']]*cellvars['Q_b2_max']

In [149]:
# PROCESS THE CBC TRAJECTORY

# gradient relative tolerance for steady state
grad_rtol=1

# go through the trajectory and find average p_b2 in the last hour of tracking each reference
ucalc_steady_states=[]
uexp_steady_states=[]
b2_steady_states=[]
b_steady_states=[]
Q_probe_steady_states=[]
Q_cifi_steady_states=[]
ref_cntr=0
for i in range(1, len(ts)):
    if((refrecord[i]!=refrecord[i-1]) or (i==len(ts)-1)):
        # find the time points for the last hour of tracking
        last_hour_indices=np.where((ts>=ts[i-1]-1) & (ts<=ts[i-1]))[0]

        # get the mean u and p_b2 in the last hour of tracking
        ucalc_mean=np.mean(ucalcrecord[last_hour_indices])
        uexp_mean=np.mean(uexprecord[last_hour_indices])
        b2_mean=np.mean(xs[last_hour_indices, modules_name2pos[controller_ctrledvar]])
        b_mean=np.mean(xs[last_hour_indices, modules_name2pos['b_mature']])
        # real burdens
        Q_probe_mean=np.mean(Q_probes[last_hour_indices])
        Q_cifi_mean=np.mean(Q_cifis[last_hour_indices])

        # before recording the value, check if the reference is somewhat steady
        grad, _ = np.polyfit(ts[last_hour_indices],
                             xs[last_hour_indices,modules_name2pos[controller_ctrledvar]],
                             deg=1)  # get a linear fit for controlled variable vs time
        # if check is passed, record the steady state
        if(abs(grad/b2_mean)<grad_rtol):
            ucalc_steady_states.append(ucalc_mean)
            uexp_steady_states.append(uexp_mean)
            b2_steady_states.append(b2_mean)
            b_steady_states.append(b_mean)
            # real burdens
            Q_probe_steady_states.append(Q_probe_mean)
            Q_cifi_steady_states.append(Q_cifi_mean)

        # move to the next reference
        ref_cntr+=1

In [150]:
# GET THE ESTIMATED BURDENS EXPERIENCED AND IMPOSED BY THE SELF-ACTIVATING SWITCH

# load data from the probe characterisation experiment
Q_probe_data = np.load('basic_Q_probe_data.npy')
Qdash_probe_data = np.load('basic_Qdash_probe_data.npy')

# create interpolation function (comment out the unwanted onptions)
# linear ND option
Q_probe_interpolator = make_interpolator_sp_lnd(Q_probe_data,
                                          normalise_u_and_y=True)
Q_cifi_interpolator = make_interpolator_sp_lnd(Qdash_probe_data,
                                              normalise_u_and_y=True)

# get the estimated burdens
Q_probe_steady_states_est=np.array(Q_probe_steady_states)
Q_cifi_steady_states_est=np.zeros_like(Q_cifi_steady_states)
for i in range(0, len(uexp_steady_states)):
    Q_probe_steady_states_est[i]=Q_probe_interpolator(ucalc_steady_states[i],b_steady_states[i])
    Q_cifi_steady_states_est[i]=Q_cifi_interpolator(ucalc_steady_states[i],b_steady_states[i])

In [151]:
# PLOT BIFURCATION DIAGRAM

# u values displayed on the plot
u_range=(0.01,11)


# for convenience of display, zero action shown as minimal action
ucalcrecord[ucalcrecord==0]=min(u_range)
uexprecord[uexprecord==0]=min(u_range)

# plot the cbc trajectory as actually observed
cbc_fig = bkplot.figure(
    frame_width=480,
    frame_height=360,
    # x_axis_type="log",
    x_axis_label="u, probe induction",
    y_axis_label="b2_mature, controlled variable",
    # x_range=(min(u_range), max(u_range)),
    # y_range=(0, max(an_bif_curve['b2_mature'])),
    title='Control action vs controlled variable',
    tools="box_zoom,pan,hover,reset,save"
)
cbc_fig.output_backend = 'svg'
# plot the cbc trajectory for calculated control actions
cbc_fig.line(x=uexprecord, y=xs[:, modules_name2pos['b2_mature']], line_width=1.5,
             line_color='blue', line_alpha=0.5,
             legend_label='cbc trajectory (real)')

# plot the CBC steady states
cbc_fig.scatter(x=uexp_steady_states, y=b2_steady_states,
                marker='circle', size=7.5,
               color='violet', legend_label='cbc steady states (real)')
# plot the estimated CBC steady states
cbc_fig.scatter(x=ucalc_steady_states, y=b2_steady_states,
                marker='circle', size=7.5,
               color='darkviolet', legend_label='cbc steady states (est)')

# legend formatting
cbc_fig.legend.label_text_font_size = "8pt"
cbc_fig.legend.location = "bottom_left"
cbc_fig.legend.click_policy = 'hide'

# plot the cbc trajectory for imposed and exerted burdens
cbc_Q_fig = bkplot.figure(
    frame_width=480,
    frame_height=360,
    x_axis_label="Q_p, probe's resource demand",
    y_axis_label="Q_cifi, self-activating switch's demand",
    # x_range=(0, 0.75),
    # y_range=(0, 0.25),
    title='Burden plot',
    tools="box_zoom,pan,hover,reset,save"
)
cbc_Q_fig.output_backend = 'svg'

# plot the cbc trajectory for imposed and exerted burdens
cbc_Q_fig.line(x=Q_probes, y=Q_cifis,
               line_width=1.5, line_color='blue', line_alpha=0.5,
               legend_label='cbc trajectory (real)')

# plot the CBC steady states
cbc_Q_fig.scatter(x=Q_probe_steady_states, y=Q_cifi_steady_states,
                marker='circle', size=7.5,
               color='violet', legend_label='cbc steady states (real)')
# plot the estimated CBC steady states
cbc_Q_fig.scatter(x=Q_probe_steady_states, y=Q_cifi_steady_states_est,
                marker='circle', size=7.5,
               color='darkviolet', legend_label='cbc steady states (est)')

# legend formatting
cbc_Q_fig.legend.label_text_font_size = "8pt"
cbc_Q_fig.legend.location = "top_right"
cbc_Q_fig.legend.click_policy = 'hide'


# show plot
bkplot.show(bklayouts.grid([[cbc_fig, cbc_Q_fig]]))

In [152]:
# CDC PAPER - FIGURE 1 B
# plot the steady states as actually observed
intro_b_fig = bkplot.figure(
    frame_width=100,
    frame_height=100,
    x_axis_label="Resource demand\nfrom CRi",
    y_axis_label="MOI output",
    x_range=(0.0, 0.6),
    y_range=(0, 5e5),
    tools="box_zoom,pan,hover,reset,save"
)
intro_b_fig.output_backend = 'svg'

# plot the characterisation outcomes
intro_b_fig.scatter(x=Q_probe_steady_states, y=b2_steady_states,
                    marker='circle', size=5, color='#bb3385ff',
                    legend_label='Observations')

# legend formatting
intro_b_fig.legend.location = "top_right"
intro_b_fig.legend.label_text_font_size = "7pt"
intro_b_fig.legend.padding=0
intro_b_fig.legend.margin=2
intro_b_fig.legend.spacing=0
intro_b_fig.legend.glyph_width=5
intro_b_fig.legend.click_policy = 'hide'

# axis formatting
intro_b_fig.yaxis.formatter=bkmodels.PrintfTickFormatter(format="%4.0e")
intro_b_fig.yaxis.ticker=bkmodels.BasicTicker(desired_num_ticks=6)
intro_b_fig.xaxis.axis_label_text_font_size = "7pt"
intro_b_fig.yaxis.axis_label_text_font_size = "7pt"
intro_b_fig.xaxis.axis_label_text_color = "black"
intro_b_fig.yaxis.axis_label_text_color = "black"
intro_b_fig.xaxis.major_label_text_font_size = "7pt"
intro_b_fig.yaxis.major_label_text_font_size = "7pt"
intro_b_fig.xaxis.major_label_text_color = "black"
intro_b_fig.yaxis.major_label_text_color = "black"

bkplot.show(intro_b_fig)

In [153]:
# CDC PAPER - FIGURE 4A
# plot the steady states as actually observed
cbc_a_fig = bkplot.figure(
    frame_width=125,
    frame_height=75,
    x_axis_label="Time [h]",
    y_axis_label="Prot. conc. [nM]",
    x_range=(0, ts[-1] - par['t_burn_in']),
    y_range=(0, 1.2e6),
    tools="box_zoom,pan,hover,reset,save"
)
cbc_a_fig.output_backend = 'svg'

# plot the references
cbc_a_fig.line(x=ts - par['t_burn_in'], y=refrecord,
               line_width=1.5, color='black',
               legend_label='ref')
# plot the module-of-interest outputs
cbc_a_fig.line(x=ts - par['t_burn_in'], y=xs[:, modules_name2pos['b2_mature']],
               line_width=1.5, color='#bb3385ff',
               legend_label='ymoi')

# plot probe outputs
cbc_a_fig.line(x=ts - par['t_burn_in'], y=xs[:, modules_name2pos['b_mature']],
               line_width=1.5, color='#48d1ccff',
               legend_label='yprobe')

# legend formatting
cbc_a_fig.legend.location = "top_left"
cbc_a_fig.legend.label_text_font_size = "6pt"
cbc_a_fig.legend.padding = 0
cbc_a_fig.legend.margin = 1
cbc_a_fig.legend.spacing = 3
cbc_a_fig.legend.glyph_width = 3
cbc_a_fig.legend.orientation = "horizontal"
cbc_a_fig.legend.click_policy = 'hide'

# axis formatting
cbc_a_fig.yaxis.formatter = bkmodels.PrintfTickFormatter(format="%4.1e")
cbc_a_fig.yaxis.ticker = bkmodels.FixedTicker(ticks=[0.0, 4e5, 8e5, 1.2e6])
cbc_a_fig.xaxis.axis_label_text_font_size = "6pt"
cbc_a_fig.yaxis.axis_label_text_font_size = "6pt"
cbc_a_fig.xaxis.axis_label_text_color = "black"
cbc_a_fig.yaxis.axis_label_text_color = "black"
cbc_a_fig.xaxis.major_label_text_font_size = "6pt"
cbc_a_fig.yaxis.major_label_text_font_size = "6pt"
cbc_a_fig.xaxis.major_label_text_color = "black"
cbc_a_fig.yaxis.major_label_text_color = "black"

# CDC PAPER - FIGURE 4B
# plot the steady states as actually observed
cbc_b_fig = bkplot.figure(
    frame_width=125,
    frame_height=75,
    x_axis_label="Time [h]",
    y_axis_label="Probe input [nM]",
    x_range=(0, ts[-1] - par['t_burn_in']),
    y_range=(0, 15),
    tools="box_zoom,pan,hover,reset,save"
)
cbc_b_fig.output_backend = 'svg'

# plot the inputs
cbc_b_fig.line(x=ts - par['t_burn_in'], y=ucalcrecord,
               line_width=1.5, color='#48d1ccff')

# axis formatting
cbc_b_fig.yaxis.ticker = bkmodels.FixedTicker(ticks=[0.0, 5, 10, 15])
cbc_b_fig.xaxis.axis_label_text_font_size = "6pt"
cbc_b_fig.yaxis.axis_label_text_font_size = "6pt"
cbc_b_fig.xaxis.axis_label_text_color = "black"
cbc_b_fig.yaxis.axis_label_text_color = "black"
cbc_b_fig.xaxis.major_label_text_font_size = "6pt"
cbc_b_fig.yaxis.major_label_text_font_size = "6pt"
cbc_b_fig.xaxis.major_label_text_color = "black"
cbc_b_fig.yaxis.major_label_text_color = "black"

# CBC PAPER - FIGURE 4C
# plot the steady states as actually observed
cbc_c_fig = bkplot.figure(
    frame_width=100,
    frame_height=100,
    x_axis_label="Probe input [nM]",
    y_axis_label="MOI output [nM]",
    y_range=(0, 5e5),
    tools="box_zoom,pan,hover,reset,save"
)
cbc_c_fig.output_backend = 'svg'

# plot theCBC trajectory
cbc_c_fig.line(x=ucalcrecord, y=xs[:, modules_name2pos['b2_mature']], line_width=1.5,
             line_color='blue', line_alpha=0.5,
             legend_label='trajectory')
# plot the characterisation outcomes
cbc_c_fig.scatter(x=ucalc_steady_states, y=b2_steady_states,
                    marker='circle', size=2.5, color='#bb3385ff',
                    legend_label='steady states')

# legend formatting
cbc_c_fig.legend.location = "top_right"
cbc_c_fig.legend.label_text_font_size = "6pt"
cbc_c_fig.legend.padding=0
cbc_c_fig.legend.margin=2
cbc_c_fig.legend.spacing=0
cbc_c_fig.legend.glyph_width=5
cbc_c_fig.legend.click_policy = 'hide'

# axis formatting
cbc_c_fig.yaxis.formatter=bkmodels.PrintfTickFormatter(format="%4.0e")
cbc_c_fig.yaxis.ticker=bkmodels.BasicTicker(desired_num_ticks=6)
cbc_c_fig.xaxis.axis_label_text_font_size = "6pt"
cbc_c_fig.yaxis.axis_label_text_font_size = "6pt"
cbc_c_fig.xaxis.axis_label_text_color = "black"
cbc_c_fig.yaxis.axis_label_text_color = "black"
cbc_c_fig.xaxis.major_label_text_font_size = "6pt"
cbc_c_fig.yaxis.major_label_text_font_size = "6pt"
cbc_c_fig.xaxis.major_label_text_color = "black"
cbc_c_fig.yaxis.major_label_text_color = "black"

# FIGURE 4D - RESOURCE-AWARE BIFURCATION DIAGRAM
# plot the resource demand bifurcation curve, determined analytically and estimated by CBC
cbc_d_fig = bkplot.figure(
    frame_width=100,
    frame_height=100,
    x_axis_label="Q\'moi, probe resource\ndemand",
    y_axis_label="Qmoi, MOI resource\ndemand",
    x_range=(0, 0.75),
    y_range=(0, 0.15),
    tools="box_zoom,pan,hover,reset,save"
)
cbc_d_fig.output_backend = 'svg'
# plot the estimated CBC steady states
cbc_d_fig.line(x=Q_probe_steady_states_est, y=Q_cifi_steady_states_est,
                    line_width=1.5, color='#bb3385ff',
                    legend_label='CBC est.')

# legend formatting
cbc_d_fig.legend.location = "top_right"
cbc_d_fig.legend.label_text_font_size = "6pt"
cbc_d_fig.legend.padding=0
cbc_d_fig.legend.margin=2
cbc_d_fig.legend.spacing=0
cbc_d_fig.legend.glyph_width=5
cbc_d_fig.legend.click_policy = 'hide'

# axis formatting
cbc_d_fig.xaxis.ticker=bkmodels.FixedTicker(ticks=[0, 0.25, 0.5, 0.75])
cbc_d_fig.yaxis.ticker=bkmodels.FixedTicker(ticks=[0, 0.05, 0.1, 0.15])
cbc_d_fig.xaxis.axis_label_text_font_size = "6pt"
cbc_d_fig.yaxis.axis_label_text_font_size = "6pt"
cbc_d_fig.xaxis.axis_label_text_color = "black"
cbc_d_fig.yaxis.axis_label_text_color = "black"
cbc_d_fig.xaxis.major_label_text_font_size = "6pt"
cbc_d_fig.yaxis.major_label_text_font_size = "6pt"
cbc_d_fig.xaxis.major_label_text_color = "black"
cbc_d_fig.yaxis.major_label_text_color = "black"


bkplot.show(bklayouts.grid([[cbc_a_fig, cbc_c_fig],
                            [cbc_b_fig, cbc_d_fig]]))